# RAG without UI

# 4.1 — Retrieval + Generation (single shot)

Goal: Combine retrieval with an LLM call. Encodes the query, retrieves top-k
snippets, builds a focused context window, and asks the model to answer using
only that context.

### Step 1 — Load `.env`, import paths, and create the OpenAI client

Ensures the API key is read from environment variables (`.env`) and sets up
project paths for consistent file locations.

In [8]:
%pip install -q python-dotenv openai sentence-transformers faiss-cpu
from dotenv import load_dotenv, find_dotenv; load_dotenv(find_dotenv())

import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from src.paths import RAW, DOCS_LIST, INDEX

from openai import OpenAI
client = OpenAI()

from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


### Step 2 — Prepare corpus and retrieval components

Loads filenames + texts from `RAW`, initializes `all-MiniLM-L6-v2`, and opens the
persisted FAISS `INDEX` for fast nearest-neighbor search.

In [9]:
names = [ln.strip() for ln in Path(DOCS_LIST).read_text(encoding="utf-8").splitlines() if ln.strip()]
doc_paths = [RAW / n for n in names]
docs_text = [p.read_text(encoding="utf-8") for p in doc_paths]

### Step 3 — Encode query, search FAISS, and build the context window

Encodes the query with normalized embeddings, searches FAISS (cap `k` at
`index.ntotal`), filters valid indices, and concatenates the retrieved snippets
into a single context string.

In [10]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")
index   = faiss.read_index(str(INDEX))

print("Docs:", names)
print("Index ntotal:", index.ntotal)

Docs: ['clinical_demo.md']
Index ntotal: 1


### Step 4 — Call the model with context and return the answer + sources

Sends the context and question to the LLM with low temperature for factuality.
Appends a `Sources:` line derived from the filenames associated with the hits.

In [11]:
q = "What A1c level diagnoses type 2 diabetes?"
qv = encoder.encode([q], normalize_embeddings=True).astype("float32")
D, I = index.search(qv, min(3, index.ntotal))
print("Top hits:", [names[i] for i in I[0] if 0 <= i < len(docs_text)])

Top hits: ['clinical_demo.md']
